##Імпорт бібліотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import pickle


Завантажуємо датасет як датафрейм методом read_csv() з pandas



In [17]:
df = pd.read_csv("sample_data/internet_service_churn.csv")
df.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [18]:
df.describe()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
count,7.227400e+04,72274.000000,72274.000000,72274.000000,72274.000000,50702.000000,72274.000000,71893.000000,71893.000000,72274.000000,72274.000000
mean,8.463182e+05,0.815259,0.334629,2.450051,18.942483,0.716039,0.274234,43.689911,4.192076,0.207613,0.554141
std,4.891022e+05,0.388090,0.471864,2.034990,13.215386,0.697102,0.816621,63.405963,9.818896,0.997123,0.497064
min,1.500000e+01,0.000000,0.000000,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.222165e+05,1.000000,0.000000,0.930000,13.000000,0.000000,0.000000,6.700000,0.500000,0.000000,0.000000
50%,8.477840e+05,1.000000,0.000000,1.980000,19.000000,0.570000,0.000000,27.800000,2.100000,0.000000,1.000000
75%,1.269562e+06,1.000000,1.000000,3.300000,22.000000,1.310000,0.000000,60.500000,4.800000,0.000000,1.000000
max,1.689744e+06,1.000000,1.000000,12.800000,406.000000,2.920000,19.000000,4415.200000,453.300000,7.000000,1.000000


Замінимо пусті значення і створимо прапорці пропусків

In [19]:
median_remaining = df["reamining_contract"].median()
df["reamining_contract"] = df["reamining_contract"].fillna(median_remaining)

df["download_avg_missing"] = df["download_avg"].isna().astype(int)
df["upload_avg_missing"] = df["upload_avg"].isna().astype(int)

download_median = df["download_avg"].median()
upload_median = df["upload_avg"].median()

df["download_avg"] = df["download_avg"].fillna(download_median)
df["upload_avg"] = df["upload_avg"].fillna(upload_median)

print("median_remaining", median_remaining)
print("download_median", download_median)
print("upload_median", upload_median)

median_remaining 0.57
download_median 27.8
upload_median 2.1


In [20]:
df.describe()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn,download_avg_missing,upload_avg_missing
count,7.227400e+04,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000,72274.000000
mean,8.463182e+05,0.815259,0.334629,2.450051,18.942483,0.672450,0.274234,43.606146,4.181047,0.207613,0.554141,0.005272,0.005272
std,4.891022e+05,0.388090,0.471864,2.034990,13.215386,0.587683,0.816621,63.249082,9.794153,0.997123,0.497064,0.072415,0.072415
min,1.500000e+01,0.000000,0.000000,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.222165e+05,1.000000,0.000000,0.930000,13.000000,0.070000,0.000000,6.800000,0.500000,0.000000,0.000000,0.000000,0.000000
50%,8.477840e+05,1.000000,0.000000,1.980000,19.000000,0.570000,0.000000,27.800000,2.100000,0.000000,1.000000,0.000000,0.000000
75%,1.269562e+06,1.000000,1.000000,3.300000,22.000000,1.040000,0.000000,60.200000,4.800000,0.000000,1.000000,0.000000,0.000000
max,1.689744e+06,1.000000,1.000000,12.800000,406.000000,2.920000,19.000000,4415.200000,453.300000,7.000000,1.000000,1.000000,1.000000


Прибираємо зайву колонку id (вона не несе корисної для передбачення інформаціх) і churn - цільову змінну.

Розбиваємо датасет на навчальні і тестові дані методом train_test_split з sklearn у пропорції 80% тренувальних і 20% тестових даних.

Додаємо параметр stratify=y, щоб відток клієнтів мав однаковий розподіл в тренуванні і тесті.

In [21]:
df = df.drop("id", axis=1)

X = df.drop("churn", axis=1)
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

##Визначимо діапазон можливих параметрів для моделі

In [22]:
param_grid = {
    "n_estimators": [150, 300],
    "max_depth": [10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt"],
    "class_weight": ["balanced"]
}


##Підбираємо найкращі параметри за допомогою GridSearchCV для RandomForestClassifier.

Щоб модель була стабільною і уникла перенавчання, крос-валідацію робимо, розбиваючи вхідні дані на 5 частин.

Максимізуємо метрику f1, яка добре працює на нерівних класах (у випадку оцінки відтоку великий дисбаланс класів).

Використовуємо усі доступні ядра процесору, щоб пришвидшити навчання.

Виводимо кожну ітерацію.

In [23]:
rf = RandomForestClassifier(random_state=42)

grid = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("\nBest score:", grid.best_score_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'class_weight': 'balanced', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}

Best score: 0.9446393202268103


## Аналіз найкращих параметрів

* 'class_weight': 'balanced' - автоматично компенсує
дисбаланс класів

* 'max_depth': 20 - глибина дерев не надто маленька, не надто велика (менше 10 - недонавчання, понад 25–30 - переобучення, 20 - ідеальна середина)

* 'max_features': 'sqrt' - стандарт для RF у задачах класифікації. Працює швидко і стабільно

* 'min_samples_leaf': 1 - вузол не буде ділитися, якщо менше 5 спостережень. Зменшує переобучення, але практично не шкодить точності

* 'min_samples_split': 5 - галуження не надто суворе - модель захоплює найдрібніші патерни

* 'n_estimators': 300 - кількість дерев (менше 150 - менш стабільно, більше 400 - довго, а приріст малий,
300 - золота середина)


# Навчання моделі з найкращими параметрами

In [24]:
best_rf_model = grid.best_estimator_
best_rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_split=5, n_estimators=150, random_state=42)

# Оцінка моделі

In [25]:
y_pred = best_rf_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      0.95      0.93      6445
           1       0.95      0.93      0.94      8010

    accuracy                           0.94     14455
   macro avg       0.94      0.94      0.94     14455
weighted avg       0.94      0.94      0.94     14455

[[6093  352]
 [ 546 7464]]


**Точність** моделі склала 0.94. *F1-score* по класах 0.93–0.94. Це високий результат, особливо для задач відтоку клієнтів, де зазвичай є дисбаланс класів.

* Для класу 0 (клієнт НЕ йде) **precision** = 0.92. Це означає, коли модель каже, що клієнт НЕ піде, у 92% випадків це правда. **recall** = 0.95. Тобто з усіх клієнтів, які не йдуть, модель знаходить 95%.

* Для класу 1 (клієнт піде) **precision** = 0.96. Якщо модель каже, що клієнт піде — це правда в 96% випадків. **recall** = 0.93. З усіх реальних клієнтів, які підуть, модель знаходить 93%.

По **матриці плутанини** бачимо 300+ помилок коли клієнт не піде, а модель каже що піде

І 500+ помилок, коли клієнт піде, а модель каже що НЕ піде.

Відносно до 6000 і 7000 правильних відповідей ці відхилення є допустимими.

**Загальний висновок**: модель може використовуватися і показувати високу точність на реальних завданнях компанії.

Подивимося наочно на 10 випадків передбачення, бачимо, що модель не зробила жодної помилки

In [27]:
# створити таблицю з 10 першими прикладами
results = pd.DataFrame({
    "real": y_test[:10].values,
    "predicted": y_pred[:10]
})

# додати вхідні фічі
results = pd.concat([X_test.reset_index(drop=True).iloc[:10], results], axis=1)

results

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,download_avg_missing,upload_avg_missing,real,predicted
0,1,0,2.39,18,0.57,0,3.3,0.1,0,0,0,1,1
1,1,0,1.09,23,0.57,0,6.8,0.5,0,0,0,1,1
2,1,1,0.02,0,1.95,0,27.8,2.1,0,1,1,0,0
3,1,1,2.01,21,0.57,0,0.0,0.0,0,0,0,1,1
4,0,0,0.02,3,0.00,0,5.9,0.3,0,0,0,1,1
5,1,0,5.99,29,0.57,0,217.3,8.5,0,0,0,0,0
6,0,0,3.28,0,0.57,0,5.4,9.4,4,0,0,1,1
7,1,1,5.93,21,0.43,1,78.7,4.4,0,0,0,0,0
8,1,0,6.52,19,1.93,1,66.9,3.6,0,0,0,0,0
9,1,1,2.84,15,1.08,1,36.3,2.5,0,0,0,0,0


# Збережемо модель для використанні у інтеграції

In [28]:
with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(best_rf_model, f)


In [30]:
import json

rf_medians = {
    "remaining_contract_median": float(median_remaining),
    "download_median": float(download_median),
    "upload_median": float(upload_median),
}
with open("rf_medians.json", "w") as f:
    json.dump(rf_medians, f)

In [31]:
report = classification_report(y_test, y_pred, output_dict=True)

with open("rf_metrics.json", "w") as f:
    json.dump(report, f, indent=4)
